In [1]:
!nvidia-smi

Sat Nov 20 18:47:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 54%   50C    P8    46W / 370W |   1581MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
|  0%   

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[1], device_type='GPU')

2.5.0


## Model Ensembling 

In [3]:
pos = 27297
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.61
Weight for class 1: 2.68


In [4]:
BATCH_SIZE = 32

record_file_train = 'copd_train_new.tfrecords'
train_dataset = (tf.data.TFRecordDataset(
    record_file_train, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(total)
.batch(BATCH_SIZE))

record_file_val = 'copd_val_new.tfrecords'
val_dataset = (tf.data.TFRecordDataset(
    record_file_val, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(36601)
.batch(BATCH_SIZE))

record_file_test = 'copd_test_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_train = get_data_label('train')
y_val = get_data_label('val')
y_test = get_data_label('test')

## Train base model

In [10]:
archi = 'IV3'

checkpoint_filepath = 'checkpoints/Recall/checkpoint_cw_InceptionV3'
monitor_ = 'val_recall'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

model = load_model_from_pretrain(archi)

# metric = tf.keras.metrics.PrecisionAtRecall(recall=0.5, name='precision_at_recall')
metric = tf.keras.metrics.Recall(name='recall')
# metric = tf.keras.metrics.Precision(name='precision')

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

y_preds_IV3 = model.predict(test_dataset)

test_CI(y_preds_IV3, y_test)
    
thresh = get_thresh(y_test, y_preds_IV3, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds_IV3 >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds_IV3, average=None))

del model, base_model, pred_layer
gc.collect()

Epoch 1/20
4576/4576 [==============================] - 489s 104ms/step - loss: 0.6651 - recall: 0.6481 - val_loss: 0.5905 - val_recall: 0.4719
Epoch 2/20
4576/4576 [==============================] - 484s 104ms/step - loss: 0.6340 - recall: 0.6737 - val_loss: 0.5775 - val_recall: 0.5518
Epoch 3/20
4576/4576 [==============================] - 478s 103ms/step - loss: 0.6151 - recall: 0.6847 - val_loss: 0.5855 - val_recall: 0.5427
Epoch 4/20
4576/4576 [==============================] - 479s 103ms/step - loss: 0.6042 - recall: 0.6918 - val_loss: 0.6861 - val_recall: 0.7387
Epoch 5/20
4576/4576 [==============================] - 478s 103ms/step - loss: 0.5857 - recall: 0.7076 - val_loss: 0.6846 - val_recall: 0.7636
Epoch 6/20
4576/4576 [==============================] - 478s 103ms/step - loss: 0.5722 - recall: 0.7115 - val_loss: 0.6774 - val_recall: 0.7444
Epoch 7/20
4576/4576 [==============================] - 479s 103ms/step - loss: 0.5496 - recall: 0.7336 - val_loss: 0.5291 - val_recall:

177

In [6]:
archi = 'Dnet121'
checkpoint_filepath = 'checkpoints/Recall/checkpoint_cw_Dnet121'
monitor_ = 'val_recall'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

model = load_model_from_pretrain(archi)

# metric = tf.keras.metrics.PrecisionAtRecall(recall=0.5, name='precision_at_recall')
metric = tf.keras.metrics.Recall(name='recall')
# metric = tf.keras.metrics.Precision(name='precision')

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

y_preds_D121 = model.predict(test_dataset)

test_CI(y_preds_D121, y_test)
    
thresh = get_thresh(y_test, y_preds_D121, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds_D121 >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds_D121, average=None))

del model, base_model, pred_layer
gc.collect()

Epoch 1/20
4576/4576 [==============================] - 705s 151ms/step - loss: 0.6521 - recall: 0.6427 - val_loss: 0.7315 - val_recall: 0.5262
Epoch 2/20
4576/4576 [==============================] - 697s 151ms/step - loss: 0.6311 - recall: 0.6647 - val_loss: 0.4921 - val_recall: 0.1703
Epoch 3/20
4576/4576 [==============================] - 697s 151ms/step - loss: 0.6144 - recall: 0.6803 - val_loss: 1.0780 - val_recall: 0.9572
Epoch 4/20
4576/4576 [==============================] - 697s 150ms/step - loss: 0.5991 - recall: 0.6959 - val_loss: 0.7683 - val_recall: 0.8440
Epoch 5/20
4576/4576 [==============================] - 697s 150ms/step - loss: 0.5802 - recall: 0.7111 - val_loss: 0.4820 - val_recall: 0.3466
Epoch 6/20
4576/4576 [==============================] - 697s 151ms/step - loss: 0.5613 - recall: 0.7245 - val_loss: 0.5890 - val_recall: 0.6387
Sample auc mean: 0.72
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.71 - 0.72]
thresh: 0.47534525
           

4

## Prediction-based ensemble

In [22]:
checkpoint_filepath = 'checkpoints/PrecisionAtRecall/checkpoint_cw_InceptionV3'
archi = 'IV3'
model1 = load_model_from_pretrain(archi)
model1.load_weights(checkpoint_filepath)

checkpoint_filepath = 'checkpoints/PrecisionAtRecall/checkpoint_cw_Dnet121'
archi = 'Dnet121'
model2 = load_model_from_pretrain(archi)
model2.load_weights(checkpoint_filepath)

y_preds_1 = model1.predict(test_dataset)
y_preds_2 = model2.predict(test_dataset)

In [23]:
from sklearn.metrics import classification_report, average_precision_score

y_preds_vote =(y_preds_1 + y_preds_2)/2
    
y_preds_vote = np.array(y_preds_vote)
    
test_CI(y_preds_vote, y_test)
    
thresh = get_thresh(y_test, y_preds_vote, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds_vote >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds_vote, average=None))

gc.collect()

Sample auc mean: 0.74
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.73 - 0.74]
thresh: 0.45092925
              precision    recall  f1-score   support

           0       0.89      0.69      0.78     36808
           1       0.34      0.65      0.45      9026

    accuracy                           0.69     45834
   macro avg       0.62      0.67      0.61     45834
weighted avg       0.78      0.69      0.71     45834

0.41804008162925604


8

## Model-based ensemble

In [14]:
checkpoint_filepath = 'checkpoints/PrecisionAtRecall/checkpoint_cw_InceptionV3'
model_path = 'saved_model/Chexpert_pretrained_256_6_labels_InceptionV3'
InceptionV3 = tf.keras.models.load_model(model_path)
pred_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='pred_layer')(InceptionV3.get_layer(InceptionV3.layers[-2].name).output)
InceptionV3 = tf.keras.Model(inputs=InceptionV3.input, outputs=pred_layer, name='model')
InceptionV3.load_weights(checkpoint_filepath)

checkpoint_filepath = 'checkpoints/PrecisionAtRecall/checkpoint_cw_Dnet121'
model_path = 'saved_model/Chexpert_pretrained_256_6_labels'
densenet121 = tf.keras.models.load_model(model_path)
pred_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='pred_layer')(densenet121.get_layer(densenet121.layers[-2].name).output)
densenet121 = tf.keras.Model(inputs=densenet121.input, outputs=pred_layer, name='model')
densenet121.load_weights(checkpoint_filepath)

InceptionV3 = tf.keras.Model(inputs=InceptionV3.input, outputs=InceptionV3.get_layer(InceptionV3.layers[-2].name).output)
densenet121 = tf.keras.Model(inputs=densenet121.input, outputs=densenet121.get_layer(densenet121.layers[-2].name).output)

features_IV3 = InceptionV3.predict(train_dataset)
features_D121 = densenet121.predict(train_dataset)

features_val_IV3 = InceptionV3.predict(val_dataset)
features_val_D121 = densenet121.predict(val_dataset)

In [16]:
checkpoint_filepath = 'checkpoints/PrecisionAtRecall/checkpoint_cw_ensemble_mlp'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

ensemble_mlp = get_ensemble_mlp()

metric = tf.keras.metrics.PrecisionAtRecall(recall=0.5, name='precision_at_recall')

ensemble_mlp.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

ensemble_mlp.fit([features_IV3, features_D121], y_train, epochs=20, shuffle=True, validation_data=([features_val_IV3, features_val_D121],  y_val), callbacks=callback, class_weight=class_weight)

Epoch 1/20
4576/4576 [==============================] - 23s 5ms/step - loss: 0.6948 - precision_at_recall: 0.1869 - val_loss: 0.6908 - val_precision_at_recall: 0.1734
Epoch 2/20
4576/4576 [==============================] - 22s 5ms/step - loss: 0.6934 - precision_at_recall: 0.1865 - val_loss: 0.6891 - val_precision_at_recall: 0.1723
Epoch 3/20
4576/4576 [==============================] - 22s 5ms/step - loss: 0.6932 - precision_at_recall: 0.1864 - val_loss: 0.6925 - val_precision_at_recall: 0.1723
Epoch 4/20
4576/4576 [==============================] - 22s 5ms/step - loss: 0.6932 - precision_at_recall: 0.1864 - val_loss: 0.6905 - val_precision_at_recall: 0.1723


In [17]:
X_test_IV3 = InceptionV3.predict(test_dataset)
X_test_D121 = densenet121.predict(test_dataset)

y_preds = ensemble_mlp.predict([X_test_IV3, X_test_D121])

test_CI(y_preds, y_test)

# plot_cm(y_preds, y_test)

thresh = get_thresh(y_test, y_preds)

print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Sample auc mean: 0.44
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.43 - 0.45]
              precision    recall  f1-score   support

           0       1.00      0.00      0.00     36808
           1       0.20      1.00      0.33      9026

    accuracy                           0.20     45834
   macro avg       0.60      0.50      0.16     45834
weighted avg       0.84      0.20      0.07     45834

0.17071281818144116


746060